# Fortune500 List Sorting
This script searches the past 30 years of Fortune500 lists for the most reoccuring companies. 
It outputs a csv file to be used in the Ticker Lookup script.

Script PFD:

1. FortuneList Sorting 
2. Ticker_Lookup(in FortuneList) 
3. Symbol_Corrections(in FortuneList) 
4. Stock_Query 
5. StockMarketCrash_Analytica

In [56]:
# import dependencies
import numpy as np
import pandas as pd

# intializing Fortune500 List lookup
start_year = 1989
end_year = 2020
year_list = list(np.arange(start_year, end_year, 1))

# creating blank dataframe to dump to
f500_df = pd.DataFrame(columns=['rank', 'company', 'year'])

# reading each fortune500 list in the year list and dumping to preliminary df
for year in year_list:
    
    # designating annual csv path
    path = f'../Resources/Fortune500_RawCSV/fortune500-{year}.csv'
    
    # reading csv into current working dataframe
    f500cw_df = pd.read_csv(path)
    
    # creating tracking column for year in current working dataframe
    f500cw_df['year'] = year
    
    # appending current working dataframe to master dataframe
    f500_df = f500_df.append(f500cw_df)
    
    f500_df.reset_index()
    

# creating sorted list of times companies appeared in the Fortune500 list
listoflists = pd.DataFrame(f500_df['company'].value_counts()).reset_index().rename(columns={'index':'company', 'company':'occurance'})

# merging sorted list back with main dataframe for company data
f500_df_sort = listoflists.merge(f500_df, on='company', how='left')

# eliminating all rows where profit was identified as NA
f500_df_sort = f500_df_sort.loc[f500_df_sort['profit ($ millions)'] != 'N.A.']

# convert profit column to float
f500_df_sort['profit ($ millions)'] = f500_df_sort['profit ($ millions)'].astype('float64')

# groupby company and taking the max values of each column
f500_df_grp = f500_df_sort.groupby('company').max().sort_values('profit ($ millions)', ascending=False)

# eliminating all occurances that fall less than 25 times in the fortune 500 lsit
f500_df_grp = f500_df_grp.loc[f500_df_grp['occurance'] >= 25]

# resetting index
f500_df_grp = f500_df_grp.reset_index()

# creating empty column(s) for ticker lookup
f500_df_grp['ticker'] = ""
f500_df_grp['company_check'] = ""

f500_df_grp

,company,occurance,profit ($ millions),rank,revenue ($ millions),year,ticker,company_check
0,Exxon Mobil,31,45220.0,4,452926.0,2019,,
1,Berkshire Hathaway,31,44940.0,295,247837.0,2019,,
2,AT&T,25,29450.0,56,170756.0,2019,,
3,Citigroup,25,24589.0,56,159229.0,2019,,
4,Microsoft,25,23150.0,250,110360.0,2019,,
...,...,...,...,...,...,...,...,...
159,OfficeMax,33,416.8,483,13270.2,2013,,
160,R.R. Donnelley & Sons,31,400.6,445,11603.0,2019,,
161,Tech Data,25,340.6,464,37239.0,2019,,
162,Graybar Electric,25,143.3,480,7202.5,2019,,


# Symbol Search (Alpha Vantage)

In [57]:
# importing dependencies
import time
import pandas as pd
from config import api_key_av
import requests

# creating hard-coded values (for ease of future expansion)
max_call = 5 #calls per minute allowed by Alpha Vantage


# creating loop counters & summary stat counters
success = 0 #success API call counter
failed = 0 #failed API call counter
pull_time = 0 #future pull time variable
counter = 0 #API call counter

df_len = f500_df_grp['company'].count()

# primary loop for ticker call list to API
for index, row in f500_df_grp.iterrows():
    
    company = row['company']

    # setting API call parameters (ticker)
    params = {
        'function' : 'SYMBOL_SEARCH',
        'keywords' : company,
        'apikey' : api_key_av
    }
    
    # base API call URL
    url = 'https://www.alphavantage.co/query?'
    
    # updating time to now
    now = time.time()
    
    # API call wait function (limited to 5 calls per minute); is utilized after counter resets from 5 to 0
    if counter == 0:
        while now < pull_time:
            now = time.time()
    
    # if statement verifying proper time and counter for being able to make a successful API call
    if pull_time < now and counter < max_call:
        
        # trying API call and skipping if failed for any reason
        try:
            
            # making API call and storing data into variable
            response = requests.get(url, params).json()
            
            # THE MEAT AND POTATOS
            # extracting symbol & company name and loading to dataframe
            symbol = response['bestMatches'][0]['1. symbol']
            company_name = response['bestMatches'][0]['2. name']
            
            f500_df_grp.loc[index, 'ticker'] = symbol
            f500_df_grp.loc[index, 'company_check'] = company_name
            
            # confirming API call
            print(f'Record {index + 1} out of {df_len} processed. SUCCESS - {company} : {symbol}')
            
            success += 1
            counter += 1
            
            # resetting counter to 0 to initiate next call limit & wait function
            if counter == max_call:
                pull_time = int(time.time()) + 62
                counter = 0
                
        # handeling failed API calls
        except:
            print(f'Record {index + 1} out of {df_len} processed. FAILED - {company}')
            
            failed += 1


            
# resorting/filtering data for output
f500_df_grp = f500_df_grp[['company', 
                           'company_check', 
                           'ticker',
                           'occurance', 
                           'profit ($ millions)', 
                           'revenue ($ millions)', 
                           'year']]


# outputting to query list (for data pull)
f500_df_grp.to_csv('../Resources/Symbol_List_AV.csv', index=False)

# terminal print out of script performance
print(f'''
DONE
------------------
Query Summary
success: {success} 
failed: {failed}
total: {index + 1}
''')

# review dataframe
f500_df_grp

Record 1 out of 164 processed. SUCCESS - Exxon Mobil : XOM
Record 2 out of 164 processed. SUCCESS - Berkshire Hathaway : BRK-A
Record 3 out of 164 processed. SUCCESS - AT&T : AT
Record 4 out of 164 processed. SUCCESS - Citigroup : C
Record 5 out of 164 processed. SUCCESS - Microsoft : MSFT
Record 6 out of 164 processed. FAILED - Wells Fargo
Record 7 out of 164 processed. FAILED - General Electric
Record 8 out of 164 processed. SUCCESS - Ford Motor : F
Record 9 out of 164 processed. SUCCESS - Pfizer : PFE
Record 10 out of 164 processed. SUCCESS - Intel : INTC
Record 11 out of 164 processed. SUCCESS - Johnson & Johnson : JNJ
Record 12 out of 164 processed. FAILED - ConocoPhillips
Record 13 out of 164 processed. SUCCESS - Procter & Gamble : PG
Record 14 out of 164 processed. FAILED - Altria Group
Record 15 out of 164 processed. SUCCESS - Merck : MRK
Record 16 out of 164 processed. SUCCESS - PepsiCo : PEP
Record 17 out of 164 processed. SUCCESS - UnitedHealth Group : UNH
Record 18 out of 1

Record 138 out of 164 processed. SUCCESS - Masco : MAS
Record 139 out of 164 processed. SUCCESS - Mattel : MAT
Record 140 out of 164 processed. SUCCESS - Whirlpool : WHR
Record 141 out of 164 processed. SUCCESS - Avon Products : AVP
Record 142 out of 164 processed. SUCCESS - Dean Foods : DF
Record 143 out of 164 processed. SUCCESS - Genuine Parts : GPC
Record 144 out of 164 processed. SUCCESS - W.W. Grainger : GWW
Record 145 out of 164 processed. SUCCESS - Ryder System : R
Record 146 out of 164 processed. FAILED - Coca-Cola Enterprises
Record 147 out of 164 processed. SUCCESS - Nordstrom : JWN
Record 148 out of 164 processed. SUCCESS - Arrow Electronics : ARW
Record 149 out of 164 processed. SUCCESS - Avnet : AVT
Record 150 out of 164 processed. SUCCESS - Foot Locker : FL
Record 151 out of 164 processed. SUCCESS - CMS Energy : CMS
Record 152 out of 164 processed. FAILED - Supervalu
Record 153 out of 164 processed. SUCCESS - Pitney Bowes : PBI
Record 154 out of 164 processed. SUCCESS - 

,company,company_check,ticker,occurance,profit ($ millions),revenue ($ millions),year
0,Exxon Mobil,Exxon Mobil Corporation,XOM,31,45220.0,452926.0,2019
1,Berkshire Hathaway,Berkshire Hathaway Inc.,BRK-A,31,44940.0,247837.0,2019
2,AT&T,Atlantic Power Corporation,AT,25,29450.0,170756.0,2019
3,Citigroup,Citigroup Inc.,C,25,24589.0,159229.0,2019
4,Microsoft,Microsoft Corporation,MSFT,25,23150.0,110360.0,2019
...,...,...,...,...,...,...,...
159,OfficeMax,,,33,416.8,13270.2,2013
160,R.R. Donnelley & Sons,R.R. Donnelley & Sons Company,RRD,31,400.6,11603.0,2019
161,Tech Data,Tech Data Corporation,TECD,25,340.6,37239.0,2019
162,Graybar Electric,,,25,143.3,7202.5,2019


# Symbol Corrections
This notebook is used to compare and update any incorrect ticker symbols found between the 2 supply methods.

1. https://www.alphavantage.co/documentation/#symbolsearch

&

2. Excel Fuzzy Match (to Yahoo Finance Stock List)

In [5]:
# import dependencies
import numpy as np
import pandas as pd

# designating source lists (path 1 = fuzzy match data, path 2 = Alpha Vantage Symbol Search)
path = '../Resources/Symbol_List_YF.csv'
path2 = '../Resources/Symbol_List_AV.csv'

# creating Dataframes from data
df1 = pd.read_csv(path, encoding="ISO-8859-1")
df2 = pd.read_csv(path2)

# merging datasets
df2 = df2.merge(df1, on='company',how='left')

# creating dataset of un-like tickers
open_items = df2.loc[df2['ticker'] != df2['Ticker']]
open_items = open_items.reset_index(drop=True)

open_items.head()

,Unnamed: 0,company,company_check,ticker,occurance,profit ($ millions),revenue ($ millions),year,Name,Ticker
0,2,AT&T,Atlantic Power Corporation,AT,25,29450.0,170756.0,2019,AT&T Inc.,T
1,5,Wells Fargo,NaN,NaN,25,23057.0,101060.0,2019,Wells Fargo & Company,WFC
2,6,General Electric,NaN,NaN,31,22208.0,183207.0,2019,General Electric Company,GE
3,11,ConocoPhillips,NaN,NaN,31,15550.0,237272.0,2019,ConocoPhillips,COP
4,13,Altria Group,NaN,NaN,31,14239.0,72944.0,2019,"Altria Group, Inc.",MO


In [6]:
# inserting manual corrections
manual_corrections={
    'AT&T' : 'T',
    'Morgan Stanley' : 'MS',
    'DuPont' : 'DD',
    'Time Warner' : 'TWX',
    'Baker Hughes' : 'BHI',
    'First Data' : 'FDC',
    'J.C. Penney' : 'JCP',
    'Ball' : 'BLL',
    'OfficeMax' : 'OMX'}

In [7]:
# updating open items to form corrections list
for index, item in open_items.iterrows():
    if open_items.loc[index, 'company'] in list(manual_corrections.keys()):
        company = open_items.loc[index, 'company']
        open_items.loc[index, 'ticker'] = manual_corrections[company]
    if pd.isna(open_items.loc[index, 'ticker']):
        open_items.loc[index, 'ticker'] = open_items.loc[index, 'Ticker']

# creating corrections list from corrected open items
corrections = open_items[['company', 'ticker', 'occurance', 'profit ($ millions)', 'revenue ($ millions)', 'year']]

correction_count = len(corrections['company'])

print(f'''
Correction List Successful
Total Items: {correction_count}
''')

# cleaning df2 into a final df to export
query_list = df2[['company', 'ticker', 'occurance', 'profit ($ millions)', 'revenue ($ millions)', 'year']]


# changing out corrections on main dataframe

total_count = 0
for index, company in df2.iterrows():
    if query_list.loc[index, 'company'] in list(corrections['company']):
        company = query_list.loc[index, 'company']
        query_list.loc[index, 'ticker'] = corrections.loc[corrections['company'] == company]['ticker'].item()
        total_count += 1
print(f'''
----------
Confirmed Corrections Made: {total_count} of {correction_count}
''')

# Output list as final ticker query list
save_path = '../Resources/Query_List.csv'
query_list.to_csv(save_path)

print(f'''
----------
Query_list.csv save successful at:{save_path}
Returning Query List..
''')

query_list


Correction List Successful
Total Items: 35


----------
Confirmed Corrections Made: 35 of 35


----------
Query_list.csv save successful at:../Resources/Query_List.csv
Returning Query List..



C:\Users\armck\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:29: FutureWarning: `item` has been deprecated and will be removed in a future version


,company,ticker,occurance,profit ($ millions),revenue ($ millions),year
0,Exxon Mobil,XOM,31,45220.0,452926.0,2019
1,Berkshire Hathaway,BRK-A,31,44940.0,247837.0,2019
2,AT&T,T,25,29450.0,170756.0,2019
3,Citigroup,C,25,24589.0,159229.0,2019
4,Microsoft,MSFT,25,23150.0,110360.0,2019
...,...,...,...,...,...,...
159,OfficeMax,OMX,33,416.8,13270.2,2013
160,R.R. Donnelley & Sons,RRD,31,400.6,11603.0,2019
161,Tech Data,TECD,25,340.6,37239.0,2019
162,Graybar Electric,NaN,25,143.3,7202.5,2019
